In [19]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from flask import Flask, request
from flask_restful import Api, Resource, reqparse

def saveData(url):
    
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'lxml') #load the requested url to bs4

    headings=[]
    contents=[]
    lists=[]
    tempP = []
    count=0
    nested = 0

    headings.append("Drug name is:") 
    contents.append(soup.find("h1").text) #extracting the drug name

    for heading in soup.find_all( ["h2","p","li"]): #loop will go through every h2 and p tags
        if(heading.name == "h2"): #this condition will executes if it finds a h2 tag
            count+=1
            if (count == 11): #after gettings the main headngs stop the loop
                break
            headings.append(heading.text) #add the founded H2 heading to the list
            if(tempP != []):
                contents.append(tempP) #add the previous founded p tags within the h2 tag to contents 
            tempP = [] # reset the temp value to incluce new p tags

        if (heading.name == "li"): #detecing a list item
            nested = 1
            if(count == 0 or count == 4): #skipping menu and ads
                nested = 0

        if(nested==1): #checking if the next p tag should be nested or not
            if (heading.name == "p"):
                lists.append(heading.text) #adding the p tag in li to a list
                nested = 0 #resetting the nested check to 0
        else:
            if(lists != []): #once all p tags in li are deteced then added to lists. This checks for the value availability of lists
                tempP.append(lists) #adding the nested p tags to a single content value
                lists = [] #resetting the lists 

            if(heading.name == "p"):
                tempP.append(heading.text) #adding normal p tags directly to tempP list

    contents.pop(1) #remove the founded description without heading
    headings.pop(10) # remove the heading founded after getting all the P tags

    df = pd.DataFrame({'subheadings':headings,'content':contents}) #add lists to the pandas data frame
    result = df.to_json(contents[0]+".json",orient="records") #print the content to a json file containing the medicine name
    result = df.to_json(orient="records")
    parsed = json.loads(result)
    json.dumps(parsed) #printing the results into jupiter view
    return parsed

def readCSV(csv):
    
    urls = pd.read_csv(csv)
    drugs = []

    for url in urls['URLs']: #read CSV line by line
        print(url)
        drugs.append(saveData(url))

In [20]:
readCSV('data.csv')

https://www.drugs.com/amoxicillin_clavulanate.html
https://www.drugs.com/abilify.html
https://www.drugs.com/mtm/abilify-maintena-injection.html
https://www.drugs.com/actos.html
